In [202]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [203]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="DataSet/Alice.txt")

('DataSet/Alice.txt', <http.client.HTTPMessage at 0x2071f1078e0>)

In [204]:
f = open("DataSet/Alice.txt",'rb')

In [205]:
sentences = []

for sentence in f: #read the line one by one
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.decode('ascii', 'ignore') #eliminate bytearrays
    
    if len(sentence) > 0:
        sentences.append(sentence)
    
f.close()

In [206]:
sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [207]:
total_data = ' '.join(sentences)
print(len(total_data))

159484


In [208]:
print(total_data[:100])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for t


In [209]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print("Size of the character set : ",vocab_size)

Size of the character set :  56


In [210]:
#This is for toy-project for understanding RNN

In [211]:
print(char_vocab)

[' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [212]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([char_vocab])
char_to_index = tokenizer.word_index
print(char_to_index)

{' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


In [213]:
convert_dict = {v:k for k,v in char_to_index.items()}
char_to_index = {}
char_to_index = {v:k-1 for k,v in convert_dict.items()}
    
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [214]:
index_to_char = {}

for key, value in char_to_index.items():
    index_to_char[value] = key

In [215]:
#curiosity  Input: curiosit  Output: uriosity  8

In [216]:
#making Dataset
seq_length = 60

n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print("Number of samples: ", n_samples)

Number of samples:  2658


In [217]:
train_X = []
train_y = []

for i in range(n_samples):
    X_sample = total_data[i * seq_length: (i+1) * seq_length]
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)
    
    #Shift one step to the right
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [221]:
print("Sample of the X Dataset: ",train_X[0])
print("Sample of the y Dataset: ",train_y[0])
print("="*85)
print("Decorded sample data for X dataset: ", [index_to_char[i] for i in train_X[0]])
print("Decorded sample data for y dataset: ", [index_to_char[i] for i in train_y[0]])

Sample of the X Dataset:  [49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
Sample of the y Dataset:  [37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
Decorded sample data for X dataset:  ['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
Decorded sample data for y dataset:  ['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', '

In [222]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)
print("Shape of the train_X: ",train_X.shape)
print("Shape of the train_y: ",train_y.shape)
print(train_X[0][0])

Shape of the train_X:  (2658, 60, 56)
Shape of the train_y:  (2658, 60, 56)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0.]


In [223]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, TimeDistributed

In [224]:
hidden_units = 256

model = Sequential()
model.add(GRU(hidden_units, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(GRU(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [225]:
model.fit(train_X, train_y, epochs=80)

Epoch 1/80
84/84 [==============================] - 3s 15ms/step - loss: 2.8743 - accuracy: 0.2324
Epoch 2/80
84/84 [==============================] - 1s 13ms/step - loss: 2.2623 - accuracy: 0.3600
Epoch 3/80
84/84 [==============================] - 1s 13ms/step - loss: 2.0831 - accuracy: 0.4023
Epoch 4/80
84/84 [==============================] - 1s 13ms/step - loss: 1.9404 - accuracy: 0.4351
Epoch 5/80
84/84 [==============================] - 1s 13ms/step - loss: 1.8212 - accuracy: 0.4667
Epoch 6/80
84/84 [==============================] - 1s 13ms/step - loss: 1.7238 - accuracy: 0.4936
Epoch 7/80
84/84 [==============================] - 1s 14ms/step - loss: 1.6402 - accuracy: 0.5151
Epoch 8/80
84/84 [==============================] - 1s 13ms/step - loss: 1.5696 - accuracy: 0.5332
Epoch 9/80
84/84 [==============================] - 1s 13ms/step - loss: 1.5066 - accuracy: 0.5515
Epoch 10/80
84/84 [==============================] - 1s 13ms/step - loss: 1.4531 - accuracy: 0.5659
Epoch 11/

In [226]:
def sentence_generation(letter, model, length):
    ix = np.array([letter])
    
    y_char = [index_to_char[ix[-1]]]
    print("Predict with ", y_char[-1], " letter number ", ix[-1])
    
    X = np.zeros((1, length, vocab_size))
    
    for i in range(length):
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
        
    return ('').join(y_char)

In [227]:
result = sentence_generation(20, model, 100)
print(result)

Predict with  6  letter number  20
1/1 [==============================] - 0s 16ms/step
6. there was nothing on it but tea. i dont see a kind of seeped for get in a great hurry. you did! sa


In [228]:
result = sentence_generation(49, model, 100)
print(result)

Predict with  t  letter number  49
1/1 [==============================] - 0s 15ms/step
the march hare,) it was at the great concert given by the queens about her and many wonderlfwn tone, 


In [229]:
result = sentence_generation(50, model, 100)
print(result)

Predict with  u  letter number  50
1/1 [==============================] - 0s 16ms/step
ut for some way of escape, and wondering whether she could get to worry about it, and then said, it w
